In [11]:
import os
import json
import re
import csv
from scipy.stats import chi2_contingency
import numpy as np
import xlrd
import pandas as pd

In [12]:
def clean_str(s):  # remove puctuation
    s = re.sub('[^0-9\w\d\s]+', '', s).replace('\r'," ").replace('\n'," ").replace('\xa0',' ')
    return s.strip().lower()

In [13]:
data_file = './../dataset_a/'

In [14]:
# store the positive and negative samples
All_Fake_Data = []
All_True_Data = []

In [15]:
# 1. get fakeNewsDatasets(celebrity)
for data_set in ['celebrityDataset/', 'fakeNewsDataset/']:
    data_path_1 = data_file + 'fakeNewsDatasets(celebrity)/' + data_set
    data_path_1_fake = data_path_1 + 'fake/'
    data_path_1_true = data_path_1 + 'legit/'
    # get fake data
    for file in os.listdir(data_path_1_fake):  
        file_path = data_path_1_fake + file
        data = open(file_path, encoding='utf-8', mode='r') 
        try:
            data = data.readlines()
        except:
            continue
        one_news = []
        for line in data:
            line = clean_str(line.strip('\n\r')).split(' ')
            one_news.extend(line)
        if one_news != ['']:
            All_Fake_Data.append(' '.join(one_news))
    # get true data
    for file in os.listdir(data_path_1_true):
        file_path = data_path_1_true + file
        data = open(file_path, encoding='utf-8', mode='r')
        data = data.readlines()
        one_news = []
        for line in data:
            line = clean_str(line.strip('\n\r')).split(' ')
            one_news.extend(line)
        if one_news != ['']:
            All_True_Data.append(' '.join(one_news))
print('all fake news length:', len(All_Fake_Data))
print('all true news length:', len(All_True_Data))
print('OK_1')

all fake news length: 490
all true news length: 490
OK_1


In [16]:
# 2. get FakeNewsNet-master
for data_set in ['BuzzFeed/', 'PolitiFact/']:
    data_path_2 = data_file + 'FakeNewsNet-master/Data/' + data_set
    data_path_2_fake = data_path_2 + 'FakeNewsContent/'
    data_path_2_true = data_path_2 + 'RealNewsContent/'
    for file in os.listdir(data_path_2_fake):
        file_path = data_path_2_fake + file
        data = open(file_path, encoding='utf-8', mode='r')
        data = json.load(data)
        news = data['text']
        news = clean_str(news).split(' ')
        if len(news) > 3:  # if the news is too short then we don't select it
            All_Fake_Data.append(' '.join(news))

    for file in os.listdir(data_path_2_true):
        file_path = data_path_2_true + file
        data = open(file_path, encoding='utf-8', mode='r')
        data = json.load(data)
        news = data['text']
        news = clean_str(news).split(' ')
        if len(news) > 3:
            All_True_Data.append(' '.join(news))
print('all fake until 0k2:', len(All_Fake_Data))
print('all true until 0k2:', len(All_True_Data))
print('OK_2')

all fake until 0k2: 699
all true until 0k2: 701
OK_2


In [17]:
# 4. get PolitiFact
politifact_path = data_file + '/politifact_data/full_politifact_data_1.csv'
data = pd.read_csv(politifact_path, sep=None, error_bad_lines=False)

claim = data["PolitiFact_explanations"]
body = data["Statement"]
claim_label = data["Truth-Rating"]


fake_index = [index for index, text in enumerate(claim_label) if int(text) == 5]
non_fake_index = [index for index, text in enumerate(claim_label) if int(text) == 0]

fake_claim = [text.lower() for index, text in enumerate(claim) if index in fake_index]
non_claim = [text.lower() for index, text in enumerate(claim) if index in non_fake_index]

fake_body = [text.lower() for index, text in enumerate(body) if index in fake_index]
non_fake_body = [text.lower() for index, text in enumerate(body) if index in non_fake_index]

for index, text in enumerate(fake_claim):
    all_text = text + " " + fake_body[index]
    text = clean_str(all_text)
    All_Fake_Data.append(text)

for index, text in enumerate(non_claim):
    all_text = text + " " + non_fake_body[index]
    text = clean_str(all_text)
    All_True_Data.append(text)



print('all fake until 0k3:', len(All_Fake_Data))
print('all true until 0k3:', len(All_True_Data))
print('OK_3')

/Users/linliu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


all fake until 0k3: 1565
all true until 0k3: 2478
OK_3


In [18]:
#### 5. get bs_detector
bs_path = data_file + 'fake.csv'
data = pd.read_csv(bs_path)
claim = data["title"]
body = data["text"]
claim_label = data["type"]

fake_index = [index for index, text in enumerate(claim_label) if text.lower() == "fake"]
fake_claim = [text.lower() for index, text in enumerate(claim) if index in fake_index]
fake_body = [text.lower() for index, text in enumerate(body) if index in fake_index]

for index, text in enumerate(fake_claim):
    all_text = text + " " + fake_body[index]
    text = clean_str(all_text)
    All_Fake_Data.append(text)

print('all fake until 0k4:', len(All_Fake_Data))
print('all true until 0k4:', len(All_True_Data))
print('OK_4')

all fake until 0k4: 1584
all true until 0k4: 2478
OK_4


In [19]:
# 6. get emergent
emergent_path = data_file + 'emergent.csv'

data = pd.read_csv(emergent_path)
claim = data["claim"]
body = data["body"]
claim_label = data["claim_label"]

fake_index = [index for index, text in enumerate(claim_label) if text.lower() == "false"]
non_fake_index = [index for index, text in enumerate(claim_label) if text.lower() == "true"]

fake_claim = [text.lower() for index, text in enumerate(claim) if index in fake_index]
non_claim = [text.lower() for index, text in enumerate(claim) if index in non_fake_index]

fake_body = [text.lower() for index, text in enumerate(body) if index in fake_index]
non_fake_body = [text.lower() for index, text in enumerate(body) if index in non_fake_index]

for index, text in enumerate(fake_claim):
    all_text = text + " " + fake_body[index]
    text = clean_str(all_text)
    All_Fake_Data.append(text)

for index, text in enumerate(non_claim):
    all_text = text + " " + non_fake_body[index]
    text = clean_str(all_text)
    All_True_Data.append(text)

print('all fake until 0k5:', len(All_Fake_Data))
print('all true until 0k5:', len(All_True_Data))
print('OK_5')

all fake until 0k5: 2135
all true until 0k5: 3215
OK_5


In [20]:
print('len All_Fake_Data:', len(All_Fake_Data))
print('len All_True_Data:', len(All_True_Data))

len All_Fake_Data: 2135
len All_True_Data: 3215


In [21]:
from sklearn.model_selection import train_test_split

In [123]:
Fake_train, Fake_test = train_test_split(All_Fake_Data ,test_size=0.1, random_state=42)
True_train, True_test = train_test_split(All_True_Data ,test_size=0.1, random_state=42)

In [133]:
f = open('./data/Fake_train','w')
for i in Fake_train:
    f.write(i+'\n')
f.close()
f = open('./data/Fake_test','w')
for i in Fake_test:
    f.write(i+'\n')
f.close()
f = open('./data/True_train','w')
for i in True_train:
    f.write(i+'\n')
f.close()
f = open('./data/True_test','w')
for i in True_test:
    f.write(i+'\n')
f.close()